# Imported Inflation Data Gathering

In [23]:
import pandas as pd

In [24]:
import_by_country = pd.read_excel('יבוא לפי ארצות באחוזים.xls')
import_by_country.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,יבוא,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,יבוא
3,year,2020.0,2019.0,2018.0,2017,2016.0,2015.0,2014.0,2013.0,2012.0,...,2009.0,2008.0,2007.0,2006.0,2005.0,2004.0,2003.0,2002.0,2001.0,2000
4,Countries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
def remove_unwanted(df, unwanted_rows):
    df = df.drop(index=unwanted_rows)
    return df

def clean_data(df, unwanted_rows):
    df = df.set_index('Unnamed: 0') # Set countries as index
    df.columns = df.loc['year'].astype('int64') # Set years as columns
    df = df.iloc[7:] # Set df to show relevant data only
    df.columns = df.columns.astype('int64') # Set columns as integers (instead of floats), for convinience
    df = remove_unwanted(df, unwanted_rows)
    
    df = df.astype('float64')
    return df

def get_eurozone_drop_members(df):
    eurozone_states_lst = pd.read_html('eurozone_members.html')[0].State.values.tolist()[:-1] # Get eurozone members
    df.loc['Eurozone']=df.loc[eurozone_states_lst].sum() # Define eurozone by sum of member countries
    df = df.drop(eurozone_states_lst) # Drop Eurozone members
    return df

unwanted_rows = ['Europe', 'European Union','European Free Trade',
                              'Association (E.F.T.A.)', 'Other European Countries','Asia',
                             'A f r i c a', 'A m e r i c a', 'North America', 'Central America',
                             'South America', 'O c e a n i a', 'Unclassified', 'Countries']


In [26]:
import_by_country = clean_data(import_by_country, unwanted_rows) # Clean Data
import_by_country

KeyError: "['Europe' 'European Union' 'European Free Trade' 'Association (E.F.T.A.)'\n 'Other European Countries' 'Asia' 'A f r i c a' 'A m e r i c a'\n 'North America' 'Central America' 'South America' 'O c e a n i a'\n 'Unclassified' 'Countries'] not found in axis"

In [16]:
import_by_country = get_eurozone_drop_members(import_by_country) 
import_by_country

ImportError: lxml not found, please install it

In [17]:
def get_largest_import_partners(df, top_n_partners):
    result_df = pd.DataFrame()
    for column in df.columns:
        temp = pd.DataFrame({column: df.nlargest(top_n_partners,[column])[column]})
        result_df = pd.concat([result_df, temp], axis=1, sort=True)

    return result_df

def get_gpmn_countries_import_share(df, countries):
    result_df = df.loc[countries]
    result_df.loc['Remaining Countries'] = df.drop(gpmn_countries).sum()
    result_df.loc['Total'] = result_df.sum(axis=0)
    return result_df

#### Get n largest import partners  

In [18]:
largerst_partners = get_largest_import_partners(df= import_by_country, top_n_partners=5)
largerst_partners

TypeError: Column 'Unnamed: 0' has dtype object, cannot use method 'nlargest' with this dtype

#### Get import share for countries in GPMN  forecast

In [11]:
gpmn_countries = ['China (1)', 'Eurozone', 'United Kingdom','India','USA', 'Brazil','Japan', 'Mexico','Russian Federation', 'South Africa']
gpmn_import_share = get_gpmn_countries_import_share(import_by_country,gpmn_countries).transpose()
gpmn_import_share.to_excel('import_share_by_gpmn_countries.xlsx')

KeyError: "None of [Index(['China (1)', 'Eurozone', 'United Kingdom', 'India', 'USA', 'Brazil',\n       'Japan', 'Mexico', 'Russian Federation', 'South Africa'],\n      dtype='object')] are in the [index]"